## Setup

### Project setup

In [14]:
if run_init:
    %pip install -U pip
    !if  [ ! -d "deep-learning-project" ] ; then git clone https://github.com/albertsgarde/deep-learning-project.git; fi
    !cd deep-learning-project && git reset --hard && git pull
    !source deep-learning-project/setup.sh deep-learning-project
    import os
    os.chdir("deep-learning-project/deep-learning")
run_init = False

In [15]:
run_init = True

In [16]:
run_init = False

### Imports

In [17]:
import matplotlib.pyplot as plt

import numpy as np
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as nn_func
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataloader import DataLoader
import IPython.display as display

import audio_samples_py as aus

import utils.plots as plots
import utils.criterion as chord_criterion
import utils.utils as utils

### Device setup

In [18]:
device, use_cuda = utils.setup_device(use_cuda_if_possible = True)

Running GPU.


## Data

In [19]:
SAMPLE_LENGTH = 1024
BATCH_SIZE = 64
SEED = 2 # Generates different data if changed. Useful to ensure that a result isn't a fluke.

possible_chord_types = [i for i in range(aus.num_chord_types())]
octave_parameters = aus.OctaveParameters(add_root_octave_probability=0.5,
        add_other_octave_probability=0.3)
parameters = aus.DataParameters(num_samples=SAMPLE_LENGTH, octave_parameters=octave_parameters, min_frequency=50, max_frequency=2000, min_frequency_std_dev=0.5, max_frequency_std_dev=3., possible_chord_types=possible_chord_types) \
    .add_sine(probability=0.5, amplitude_range=(0.1,0.2)) \
    .add_saw(probability=0.5, amplitude_range=(0.1, 0.2)) \
    .add_pulse(probability=0.5, amplitude_range=(0.1, 0.2), duty_cycle_range=(0.1, 0.9)) \
    .add_triangle(probability=0.5, amplitude_range=(0.1, 0.2)) \
    .add_noise(probability=1, amplitude_range=(0.001, 0.04)) \
    .apply_distortion(probability=0.5, power_range=(0.1, 20)) \
    .apply_normalization(probability=1)

### Generation

In [20]:
def label_to_target(label: aus.DataPointLabel):
    
    target = np.zeros(aus.num_chord_types() + 12, dtype=np.float32)
    target[label.chord_type()] = 1
    target[aus.num_chord_types() + label.note()] = 1
    return target

training_parameters, training_loader, validation_parameters, validation_loader = utils.init_synth_data(parameters, label_to_target, SEED, BATCH_SIZE)

## Neural Network

In [21]:
net = torch.jit.load("models/model1_1024_80000.pt")

if use_cuda:
    net.cuda()

## Evaluation

In [22]:
data_header = ["Base.Freq", "Min.Freq", "Max.Freq", "Tone", "Type", "Acc.Type", "Acc.Tone", "Acc", "Out.Type", "Out.Tone"]

def build_data_row(label: aus.DataPointLabel, output, target):
    if label.frequencies():
        frequencies = label.frequencies()
        min_frequency = min(frequencies)
        max_frequency = max(frequencies)
    else:
        min_frequency = None
        max_frequency = None
    
    output_chord_type = np.argmax(output[:aus.num_chord_types()])
    output_chord_tone = np.argmax(output[aus.num_chord_types():])
    accurate_chord_type = output_chord_type == label.chord_type()
    accurate_chord_tone = output_chord_tone == label.note()
    accurate = accurate_chord_type and accurate_chord_tone
    

    return [label.frequency(), min_frequency, max_frequency, label.note(), label.chord_type_name(), accurate_chord_type, accurate_chord_tone, accurate, output_chord_type, output_chord_tone]


In [23]:
criterion = chord_criterion.ChordToneLoss(aus.num_chord_types())  

In [24]:
%pip install line_profiler
%load_ext line_profiler

Note: you may need to restart the kernel to use updated packages.
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [25]:
NUM_DATA_POINTS = 100000
PRINT_PROGRESS_EVERY = 100
OUTPUT_PATH = "eval_data.csv"

data = []

net.eval()
def eval():
    for batch_signal, batch_fft, batch_target, batch_labels in utils.cycle_data_loader(training_loader):
        batch_signal = utils.to_torch(batch_signal)
        batch_fft = utils.to_torch(batch_fft)
        batch_target = utils.to_torch(batch_target)
        batch_output = net(batch_signal, batch_fft)

        for i in range(batch_signal.shape[0]):

            label = batch_labels[i]
            target = utils.to_numpy(batch_target[i,:])
            output = utils.to_numpy(batch_output[i,:])
            
            data_row = build_data_row(label, output, target) 
            data.append(data_row)

            if len(data) % PRINT_PROGRESS_EVERY == 0:
                print(f"Processed: {len(data)}", end="\r")
            if len(data) >= NUM_DATA_POINTS:
                break
        if len(data) >= NUM_DATA_POINTS:
            break
#%lprun -f eval eval()
eval()



c:\Users\alber\anaconda3\envs\torch-notebook\lib\site-packages\torch\nn\modules\module.py:1190: UserWarning: FALLBACK path has been taken inside: torch::jit::fuser::cuda::runCudaFusionGroup. This is an indication that codegen Failed for some reason.
To debug try disable codegen fallback path via setting the env variable `export PYTORCH_NVFUSER_DISABLE=fallback`
 (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\jit\codegen\cuda\manager.cpp:336.)
  return forward_call(*input, **kwargs)


KeyboardInterrupt: 

In [ ]:
import csv

with open(OUTPUT_PATH, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(data_header)
    writer.writerows(data)
